In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
import jax
import jax.numpy as jnp

print("JAX devices:", jax.devices())


JAX devices: [cuda(id=0)]


In [3]:
import jax
import jaxlib
print("JAX version:", jax.__version__)
print("JAXlib version:", jaxlib.__version__)

JAX version: 0.4.26
JAXlib version: 0.4.26


In [4]:
!pip install datasets sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
from transformers import MarianTokenizer, MarianMTModel, FlaxMarianMTModel, MarianConfig
from transformers.modeling_flax_pytorch_utils import convert_pytorch_state_dict_to_flax
import optax
import pandas as pd
from datasets import load_dataset, Dataset
import sacrebleu

In [ ]:
# Chuyển mô hình từ pytorch -> flax và lưu lại
#  from flax.traverse_util import flatten_dict, unflatten_dict
# from flax.core import freeze, unfreeze

# model_name = 'Helsinki-NLP/opus-mt-en-hu'

# # Load the PyTorch model
# pytorch_model = MarianMTModel.from_pretrained(model_name)
# pytorch_state_dict = pytorch_model.state_dict()

# # Load the Flax configuration
# config = MarianConfig.from_pretrained(model_name)

# # Initialize the Flax model
# flax_model = FlaxMarianMTModel(config)
# flax_params = unfreeze(flax_model.params)

# # Function to convert pytorch state dict to flax
# def convert_pytorch_to_flax(pytorch_state_dict, flax_params):
#     pytorch_state_dict = {k: v.numpy() for k, v in pytorch_state_dict.items()}
#     flax_params_flat = flatten_dict(flax_params)
#     flax_params_flat = {
#         k: jnp.array(pytorch_state_dict[".".join(k)])
#         if ".".join(k) in pytorch_state_dict else v
#         for k, v in flax_params_flat.items()
#     }
#     flax_params = unflatten_dict(flax_params_flat)
#     return freeze(flax_params)

# # Convert PyTorch weights to Flax
# flax_params = convert_pytorch_to_flax(pytorch_state_dict, flax_params)

# # Save the Flax model
# model_dir = './flax_marian_mt_en_hu'
# flax_model.save_pretrained(model_dir, params=flax_params)

In [7]:
# Load the Tokenizer, model
model_name = 'Helsinki-NLP/opus-mt-en-hu'
model_dir = '/content/drive/MyDrive/jax and flax/flax_marian_mt_en_hu'
model = FlaxMarianMTModel.from_pretrained(model_dir)
tokenizer = MarianTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/792k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/850k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
def preprocess_function(examples, tokenizer, max_length=128):
    if 'en' not in examples or 'hu' not in examples:
        raise ValueError("Examples must contain 'en' and 'hu' keys.")

    inputs = tokenizer(examples['en'], max_length=max_length, truncation=True, padding='max_length')
    targets = tokenizer(examples['hu'], max_length=max_length, truncation=True, padding='max_length')

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids']
    }

# Đọc dữ liệu từ file CSV
data_dir = '/content/drive/MyDrive/jax and flax/'
train_data = pd.read_csv(data_dir + 'data1000/fold_1_train.csv')
valid_data = pd.read_csv(data_dir + 'data1000/fold_1_valid.csv')
test_data = pd.read_csv(data_dir + 'data1000/test_data.csv')

# Convert data to datasets
train_dataset = Dataset.from_pandas(train_data)
valid_dataset = Dataset.from_pandas(valid_data)
test_dataset = Dataset.from_pandas(test_data)

# Preprocess the datasets
train_dataset = train_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
valid_dataset = valid_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
test_dataset = test_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)

Map:   0%|          | 0/810 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
def create_train_state(model, learning_rate):
    """Tạo trạng thái huấn luyện ban đầu."""
    class TrainState(train_state.TrainState):
        batch_stats: dict = None

    params = model.params
    tx = optax.adam(learning_rate)
    state = TrainState.create(apply_fn=model.__call__, params=params, tx=tx)
    return state

@jax.jit
def train_step(state, batch, rng_key):
    def loss_fn(params):
        logits = state.apply_fn(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            params=params,
            dropout_rng=rng_key,
            train=True
        )[0]
        labels_one_hot = jax.nn.one_hot(batch['labels'], logits.shape[-1])
        loss = optax.softmax_cross_entropy(logits, labels_one_hot).mean()
        return loss

    grads = jax.grad(loss_fn)(state.params)
    state = state.apply_gradients(grads=grads)
    return state, loss_fn(state.params)

@jax.jit
def eval_step(state, batch):
    logits = state.apply_fn(batch['input_ids'], attention_mask=batch['attention_mask'], params=state.params, train=False)[0]
    labels_one_hot = jax.nn.one_hot(batch['labels'], logits.shape[-1])
    loss = optax.softmax_cross_entropy(logits, labels_one_hot).mean()
    return loss

state = create_train_state(model, learning_rate=1e-3)

In [17]:
print(train_dataset[0].keys())

dict_keys(['id', 'en', 'hu', 'input_ids', 'attention_mask', 'labels'])


In [18]:
def collate_fn(batch, train = True):
    collated_batch = {}
    columns_to_keep = ['input_ids', 'attention_mask', 'labels']

    if train:
        for key in batch.column_names:
            if key in columns_to_keep:
                collated_batch[key] = jnp.array(batch[key])
    else:
        for key in batch.keys():
            if key in columns_to_keep:
                collated_batch[key] = jnp.array(batch[key])


    return collated_batch

def train_epoch(state, train_ds, batch_size, rng_key):
    num_batches = len(train_ds) // batch_size + int(len(train_ds) % batch_size != 0)
    total_loss = 0

    for i in range(num_batches):
        # Tạo danh sách các chỉ số cho batch hiện tại
        batch_indices = list(range(i * batch_size, min((i + 1) * batch_size, len(train_ds))))
        # Lấy batch từ tập dữ liệu
        batch = train_ds.select(batch_indices)

        # Sử dụng hàm collate_fn để xử lý dữ liệu batch
        batch = collate_fn(batch, train=True)
        state, loss = train_step(state, batch, rng_key)
        total_loss += loss

    avg_loss = total_loss / num_batches
    return state, avg_loss

def evaluate(state, valid_ds, batch_size):
    """Evaluate the model."""
    loss = 0
    for i in range(0, len(valid_ds), batch_size):
        batch = valid_ds[i:i+batch_size]
        batch = collate_fn(batch,train = False)
        loss += eval_step(state, batch)
    return loss / (len(valid_ds) / batch_size)

def calculate_bleu(state, valid_ds, batch_size):
    """Calculate BLEU score on the validation set."""
    references = []
    hypotheses = []

    num_batches = len(valid_ds) // batch_size + int(len(valid_ds) % batch_size != 0)

    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(valid_ds))
        batch = valid_ds.select(range(batch_start, batch_end))

        # Process input texts
        inputs = tokenizer(batch['en'], return_tensors='jax', padding='max_length', max_length=128, truncation=True)
        translated_tokens = model.generate(**inputs)

        # Convert tokens to numpy array if needed
        translated_tokens = jax.device_get(translated_tokens.sequences)

        # Decode the translated tokens
        translated_texts = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

        # Append results
        hypotheses.extend(translated_texts)
        references.extend(batch['hu'])

    # Prepare references for BLEU calculation
    references = [[ref] for ref in references]  # sacrebleu expects a list of reference lists

    # Calculate BLEU score
    bleu = sacrebleu.corpus_bleu(hypotheses, references)
    return bleu.score

In [21]:
from jax import random
num_epochs = 1
batch_size = 16

for epoch in range(num_epochs):
    print('Epoch', epoch+1)
    rng_key = jax.random.PRNGKey(0)
    state, train_loss = train_epoch(state, train_dataset, batch_size, rng_key)
    val_loss = evaluate(state, valid_dataset, batch_size)
    bleu_score = calculate_bleu(state, valid_dataset, batch_size)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, BLEU Score: {bleu_score:.2f}')

#save model
from flax.training.checkpoints import save_checkpoint
save_checkpoint('/content/drive/MyDrive/jax and flax/flax_marian_mt_en_hu1', state, step = 1)

Epoch 1
Epoch 1, Train Loss: 3.2572, Validation Loss: 2.9898, BLEU Score: 3.03


'/content/drive/MyDrive/jax and flax/flax_marian_mt_en_hu1/checkpoint_1'

In [ ]:
# def translate(sentence, state, tokenizer):
#     inputs = tokenizer(sentence, return_tensors='jax', padding='max_length', max_length=128, truncation=True)
#     translated_tokens = model.generate(**inputs)
#     translated_text = tokenizer.decode(translated_tokens.sequences[0], skip_special_tokens=True)
#     return translated_text

In [ ]:
# # Dịch dữ liệu test
# test_sentences = test_data['en'].tolist()
# translations = [translate(sentence, state, tokenizer) for sentence in test_sentences]

In [23]:
# Đọc checkpoint
from flax.training.checkpoints import restore_checkpoint
restored_state = restore_checkpoint('/content/drive/MyDrive/jax and flax/flax_marian_mt_en_hu1/checkpoint_1', target = state)

In [27]:
def evaluate_on_test(state, test_dataset, batch_size):
    test_loss = evaluate(state, test_dataset, batch_size)
    test_bleu = calculate_bleu(state, test_dataset, batch_size)
    print(f'Test Loss: {test_loss:.4f}, Test BLEU Score: {test_bleu:.2f}')

# Đánh giá trên tập dữ liệu test
evaluate_on_test(restored_state, test_dataset, batch_size = 32)

Test Loss: 3.6264, Test BLEU Score: 0.34
